# Exploring ODF Extraction - BP Data Analysis
## Sam Potter
## Current: 4/29/19

## Path and Imports

In [ ]:
import sys
import os

# psfdi
sys.path.extend(['C:\\Users\\potterst1\\Desktop\Repositories\Github\psfdi',
                 'C:/Users/potterst1/Desktop/Repositories/Github/psfdi'])
sys.path.extend(['/workspace/stpotter/git/bitbucket/psfdi'])

from psfdi import visualize
from psfdi import fileIO
from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import patches
from matplotlib import rc
from psfdi import utilities
from psfdi import odf
from psfdi import image_processing as imp
import seaborn as sns
import cv2
import matplotlib

from ipywidgets import *
from scipy import optimize as sciopt
from scipy.stats import beta
from scipy.stats import sem

## Magics

In [ ]:
%matplotlib inline

# Visualize Axis Confirmation Images

In [ ]:
paper_data_path = 'C:\\Users\\potterst1\\Box Sync\\Research\\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\4.28.19\\3) Paper\\Sample\\Vertical\\Green'
paper_pic = cv2.imread(os.path.join(paper_data_path, 'planar_angle_0.tiff'), -1)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(paper_pic, cmap='gray');

# BP Data

## Load Raw Intensity pSFDI Data - 360 Degrees

### Specify file paths

In [ ]:
bp_raw_path = 'C:\\Users\\potterst1\\Box Sync\\Research\\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\4.28.19\\2) pSFDI\\Sample\\Vertical\\Green'
bp_stanard_path = 'C:\\Users\\potterst1\\Box Sync\\Research\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\4.28.19\\1) Standard\\Standard\\Vertical\\Green'

### Specify spatial frequency and polarizer resolution

In [ ]:
sfx_per = 36.4679
polar_res = 4
polar_max = 360
polar_angles = np.arange(0, polar_max, polar_res)

bp_planar = [cv2.imread(os.path.join(bp_raw_path, 'planar_angle_' + str(angle) + '.tiff'), -1)for angle in polar_angles]
bp_planar = np.array(bp_planar)

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.imshow(bp_planar[0, :, :], cmap='gray');

## Load in SALS data

In [ ]:
SALS_data_path = 'C:\\Users\\potterst1\\Box Sync\\Research\\Projects\\IGA DIC pSFDI\\Experimental Data\\2D pSFDI data\\ODF Extraction\\SALS\\BP\\Results\\BP ODF 042919'

SALS_data_dict = fileIO.read_SALS(os.path.join(SALS_data_path, 'SALSA OUT TXT\\BP_ODF_42919_SALSA.txt'))

In [ ]:
sals_pd_im = cv2.imread(os.path.join(SALS_data_path, 'Images\\BP_ODF_42919_PrefD.png'), -1)
sals_pd_im = cv2.cvtColor(sals_pd_im, cv2.COLOR_BGR2RGB)

In [ ]:
fig = plt.figure(figsize=(15, 15))
plt.title('SALSA Processed PD')
plt.imshow(sals_pd_im);

In [ ]:
x = SALS_data_dict['x']
y = SALS_data_dict['y']

PD_2d = SALS_data_dict['PD']
SD_2d = SALS_data_dict['SD']

In [ ]:
# Why this? What's happening here?
PD_2d = np.flip(PD_2d, 0)
SD_2d = np.flip(SD_2d, 0)

In [ ]:
# SALS PD
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(PD_2d, cmap='hsv')
plt.title('SALS PD')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (0, 180);
im0.set_clim(colorlimits)
plt.colorbar(im0, cax=cax);

In [ ]:
# SALS SD
fig = plt.figure(figsize=(10, 10))
im0 = plt.imshow(SD_2d, cmap='jet')
plt.title('SALS SD')
ax = plt.gca()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (45, 55);
im0.set_clim(colorlimits)
plt.colorbar(im0, cax=cax);

## Feasibility Study - Raw Data

### No extra registration. This isn't completely correct, but okay for now. Can improve by multiplying warp matrix translation by numer of psfdi pixels in SALS pixel as a start.

## Set the row and column data from visualization etc

In [ ]:
row, col = 300, 425

sub_psfdi = np.zeros((7 * 7, len(polar_angles)))
sub_row = 0
for row in range(row, row + 7):
    for col in range(col, col + 7):
        temp = bp_planar[:, row, col]
        sub_psfdi[sub_row, :] = temp
        sub_row += 1

sub_psfdi_mean = np.mean(sub_psfdi, axis=0)
sub_psfdi_sem = sem(sub_psfdi, axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
plt.imshow(bp_planar[0, :, :], cmap='gray');
rect = patches.Rectangle((col, row), 7, 7, edgecolor='r', facecolor='r')
ax.add_patch(rect);

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.plot(polar_angles, sub_psfdi_mean);
plt.fill_between(polar_angles, sub_psfdi_mean + sub_psfdi_sem, sub_psfdi_mean - sub_psfdi_sem, color='gray', alpha=0.2);
plt.xlabel('Theta')
plt.ylabel('Intensity [a.u.]')
plt.title('Mean pSFDI Signal pm SD over 2 Pi');

In [ ]:
row = 15
col = 15
ydim = SALS_data_dict['PD'].shape[0]

index = col * ydim + (ydim - col)

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(20, 20))

im0 = ax0.imshow(PD_2d, cmap='hsv')
divider = make_axes_locatable(ax0)
cax0 = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (0, 180);
im0.set_clim(colorlimits)
fig.colorbar(im0, cax=cax0)
rect = patches.Rectangle((col, row), 1, 1, edgecolor='k', facecolor='k')
ax0.add_patch(rect)
ax0.set_title('SALS PD - Pixel Location in Black');

im1 = ax1.imshow(SD_2d, cmap='jet')
divider = make_axes_locatable(ax1)
cax1 = divider.append_axes('right', size='5%', pad=0.05)
colorlimits = (45, 55);
im1.set_clim(colorlimits)
fig.colorbar(im1, cax=cax1)
rect = patches.Rectangle((col, row), 1, 1, edgecolor='k', facecolor='k')
ax1.add_patch(rect)
ax1.set_title('SALS SD - Pixel Location in Black');

## Fourier Series Fit

### SALS Data

In [ ]:
odf_theta = SALS_data_dict['theta'][index, :]
odf_gamma = SALS_data_dict['odf'][index, :]

In [ ]:
an, bn, c = odf.fit_fourier(20, odf_gamma, odf_theta)
odf_fsfit = odf.compute_fourier(an, bn, c, odf_theta)

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.scatter(np.rad2deg(odf_theta), odf_gamma, marker='o', s=10, label='SALS ODF Data')
plt.plot(np.rad2deg(odf_theta), odf_fsfit, color='r', label='Fourier Series Fit');
plt.ylabel('Gamma');
plt.xlabel('Theta');
plt.legend()
plt.title('SALS ODF Data and Associated Fourer Series Fit');

In [ ]:
fig = plt.figure(figsize=(10, 10))
plt.polar(odf_theta, odf_fsfit, color='r', label='Fourier Series Fit');
plt.scatter(odf_theta, odf_gamma, s=10, label='SALS ODF Data');
plt.legend()
plt.title('SALS ODF Data and Associated Fourier Series Fit - Polar Plot');

## Compute the Primary and Secondary Directions

In [ ]:
theta1, theta2 = odf.structural_eigenval_thetas(an[0], bn[0])

In [ ]:
print('Theta I (deg): {}'.format(theta1))
print('Theta II (deg): {}'.format(theta2))

In [ ]:
fig = plt.figure(figsize=(15, 10))
plt.scatter(np.rad2deg(odf_theta), odf_gamma, s=10, label='SALS ODF Data');
plt.plot(np.rad2deg(odf_theta), odf_fsfit, color='r', label='Fourier Series Fit')
plt.plot((theta1 + 180) * np.ones(len(odf_theta)), np.linspace(0.12, 0.20, len(odf_theta)), color='k', label='Theta I')
plt.plot(theta2 * np.ones(len(odf_theta)), np.linspace(0.12, 0.20, len(odf_theta)), color='g', label='Theta II')
plt.plot((theta2 + 180) * np.ones(len(odf_theta)), np.linspace(0.12, 0.20, len(odf_theta)), color='g', label='Theta II')
plt.xlabel('Theta')
plt.ylabel('Gamma')
plt.legend();
plt.title('SALS ODF Data and Associated Fourier Series Fit - Primary Directions from Structural Tensor');

In [ ]:
theta1_r = np.deg2rad(theta1)
theta2_r = np.deg2rad(theta2)
fig = plt.figure(figsize=(10, 10))
plt.polar(odf_theta, odf_fsfit, color='r', label='Fourier Series Fit');
plt.scatter(odf_theta, odf_gamma, s=10, label='SALS ODF Data');
plt.polar([theta1_r, theta1_r + np.pi], [0.20, 0.20], color='k', label='Theta I')
plt.polar([theta2_r, theta2_r + np.pi], [0.20, 0.20], color='g', label='Theta II')
plt.legend()
plt.title('SALS Data and Fourier Series Fit - Primary Directions on Polar Plot');

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(15, 15), subplot_kw=dict(projection='polar'))
ax0.plot(np.deg2rad(polar_angles), sub_psfdi_mean, linestyle='--', marker='o', color='g',
         label='Mean Fiber Intensity');
ax0.set_title('Mean pSFDI Raw Fiber Distribution Intensity In SALS Beam ROI - Planar Data');
ax0.set_ylim([0.975 * np.min(sub_psfdi_mean), 1.025 * np.max(sub_psfdi_mean)])

ax1.plot(odf_theta, odf_gamma, color='r', label='SALS ODF');
ax1.set_ylim([0, 0.25])
ax1.set_title('SALS ODF');